In [23]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sys
import os
from serial import Serial
import time
from goto import with_goto

In [32]:
# fig, ax = plt.subplots()
# line, = ax.plot(np.random.rand(10))
# ax.set_ylim(-5000, 5000)
# xdata, ydata = [0]*100, [0]*100
myPort = Serial("COM3",500000)
# raw.open()

#bufferSize is the length of the input buffer
bufferSize = 256
idx = 0


def read_serial():
    carve = 0
    highByte = 0
    lowByte = 0
    buffer = np.zeros(bufferSize)
    while (myPort.in_waiting < bufferSize):
        pass
    for i in range(bufferSize):
        carve = ord(myPort.read())
        while (carve != ord(b'\xff')):
            carve = ord(myPort.read())
        while (myPort.in_waiting < 2):
            pass
        highByte = ord(myPort.read())
        lowByte = ord(myPort.read())
        buffer[i] = highByte << 8 | lowByte
    return buffer

# def read_simulation():
#     carve = 0
#     highByte = 0
#     lowByte = 0
#     buffer = np.zeros(bufferSize)
#     time.sleep(0.25)
#     buffer = np.random.randn(256)
    
#     return buffer

# def update(data):
#     line.set_ydata(data)
#     return line,

# def run(data):
#     t,y = data
#     line.set_data(t, y)
#     return line,

# def data_gen():
#     pltx = np.array([])
#     plty = np.array([])
#     global idx
#     while True:
#         try:
#             dat = read_serial()
        
#         except:
#             dat = np.zeros(bufferSize)

#         plt.axis([idx-20480, idx + bufferSize, -100, 700])
#         for i in range(bufferSize):
#             pltx=np.append(pltx,idx)
#             idx += 1
#         # print(dat)
#         plty = np.concatenate((plty,dat))
#         pltx = pltx[-20480:]
#         plty = plty[-20480:]
#         yield pltx, p20lty


# FILE = sys.argv[1] + '.txt'
# with open(FILE, 'w'):
#     pass
# f = open(FILE, 'a')
# ani = animation.FuncAnimation(fig, run, data_gen, interval=0, blit=True)
# plt.show()
# f.close()

In [33]:
class cutGesture: 
    inputData = []
    currentWindow = []
    gestureWindow = []
#     gestureWindowNum = 4096
#     slidingWindowNum = 512
    deviationThreshold = 100
    lastState = '0'
    state = '0'
#     scanTime = 0.1
    flagForSliding = 0
    LocationFlag = 0
    
    def __init__(self, inputdata):
        self.inputData = inputdata
    
#     def getBaseLine(self):
#         baseline = 0
#         for i in range(0, self.slidingWindowNum, 1):
#             baseline += self.inputData[i]
#         baseline = baseline / self.slidingWindowNum
#         return baseline
    
    def checkState(self):
        current_deviation = np.var(self.currentWindow)
        if current_deviation > self.deviationThreshold:
            self.lastState = self.state
            self.state = '1'
        else:
            self.lastState = self.state
            self.state = '0'
    
    def getGesture(self):
#         tempLocationFlag = self.flagForSliding * self.slidingWindowNum


        self.LocationFlag = self.LocationFlag + 256
        self.currentWindow = self.inputData[self.LocationFlag:self.LocationFlag + 256]
#         tempLocationFlag = self.flagForSliding * self.slidingWindowNum
        
        if self.state == '1' or (self.state == '0' and self.lastState == '0'):
            self.checkState()
        elif self.state == '0' and self.lastState == '1':
            self.checkState()
            if(self.state == '0'):
                self.gestureWindow = self.inputData[self.LocationFlag - 4096:self.LocationFlag]
                return self.gestureWindow
        return []
    
#     def getGestureSimplified(self):
#         tempLocationFlag = self.flagForSliding * self.slidingWindowNum
#         self.currentWindow = self.inputData[tempLocationFlag:tempLocationFlag + self.slidingWindowNum]
#         self.flagForSliding = self.flagForSliding + 1
#         current_deviation = np.var(self.currentWindow)
#         if current_deviation > self.deviationThreshold:
#             self.gestureWindow = self.inputData[tempLocationFlag - self.slidingWindowNum:tempLocationFlag - self.slidingWindowNum + 4096]
#             return self.gestureWindow
#         return []
        

In [34]:
def getGesture(flagForTime, cutgesture):
    gesture = []
    while 1:
        buffer = read_serial()
#         flagForTime = flagForTime + 1
        cutgesture.inputData = np.concatenate((cutgesture.inputData, buffer), axis=None)
#         if(flagForTime % 2 == 0):
        gesture = cutgesture.getGesture()
        if len(gesture) > 0:
            yield gesture


In [35]:
@with_goto
def a():
    flagForTime = 1
    inputdata = read_serial()
    cutgesture = cutGesture(inputdata)
    
    g = getGesture(flagForTime, cutgesture)
    
    label .nextForGetGesture
    gesture = next(g)
    print("gesture-------------------")
    print(len(gesture))
    print(gesture[10:30])
    goto .nextForGetGesture

a()

gesture-------------------
512
[421. 419. 418. 416. 418. 406. 420. 417. 414. 413. 412. 411. 412. 410.
 411. 409. 410. 409. 408. 413.]
gesture-------------------
4096
[465. 465. 466. 465. 466. 465. 462. 462. 465. 466. 465. 466. 462. 471.
 464. 463. 466. 466. 467. 465.]
gesture-------------------
4096
[422. 422. 422. 423. 422. 419. 419. 422. 423. 422. 423. 419. 428. 422.
 422. 422. 423. 424. 422. 421.]
gesture-------------------
4096
[400. 402. 400. 399. 401. 400. 400. 399. 399. 400. 398. 396. 399. 400.
 399. 398. 399. 400. 388. 404.]
gesture-------------------
4096
[421. 419. 418. 422. 422. 420. 421. 421. 420. 422. 420. 419. 418. 421.
 421. 421. 422. 418. 425. 419.]
gesture-------------------
4096
[417. 419. 418. 418. 419. 417. 421. 409. 426. 419. 418. 419. 416. 418.
 420. 419. 418. 418. 418. 418.]


TypeError: ord() expected a character, but string of length 0 found

In [31]:
myPort.close()